In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import statistics
import sklearn
import sklearn.metrics as sm
import lightgbm as lgb

from sklearn import metrics
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score

from sklearn.naive_bayes import MultinomialNB
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import roc_auc_score
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier
from lightgbm import LGBMClassifier
from lightgbm import LGBMRegressor

pd.options.display.max_rows = 1500
pd.options.display.max_columns = 130

In [ ]:
train_set = pd.read_csv("../input/tabular-playground-series-sep-2021/train.csv")
test_set = pd.read_csv("../input/tabular-playground-series-sep-2021/test.csv")
train_set.head()

In [ ]:
train_set.shape

# Null Values

In [ ]:
train_set.isnull().sum().sort_values(ascending =False)

In [ ]:
target_category = train_set["claim"].unique()
target_category=list(map(str,target_category))
print(target_category)


# Fill Null Values with Mean Values of each feature

In [ ]:
train_set = train_set.fillna(train_set.mean()) 

In [ ]:
train_set.isnull().sum().sort_values(ascending=False)

# Data Visualization

In [ ]:
train_set.hist(figsize=(30,30), bins =120)
plt.show()

In [ ]:
train_set['claim'].value_counts().plot(kind = 'bar', figsize = (5, 5))
plt.xlabel('Claim')
plt.ylabel('Count')
plt.show()

In [ ]:
train_set.head()

In [ ]:
y= train_set['claim']
x= train_set.drop(columns=['claim','id'])
print(len(x.columns))


In [ ]:
X_train, X_test, Y_train, Y_test = train_test_split(x,y, test_size = 0.3, random_state = 60,shuffle=True)
print(len(X_train))
print(len(X_test))

# 1. XGB Classifier

In [ ]:
from xgboost import XGBClassifier
xgb = XGBClassifier()

xgb.fit(X_train, Y_train)

test_predict = xgb.predict(X_test)

# XGB AUC & ROC Curve

In [ ]:
y_pred_proba = xgb.predict_proba(X_test)[::,1]
fpr, tpr, _ = metrics.roc_curve(Y_test,  y_pred_proba)
auc = metrics.roc_auc_score(Y_test, y_pred_proba)
print(auc)
plt.plot(fpr,tpr,label="data 1, auc="+str(auc))
plt.legend(loc=4)
plt.show()


# LightGBM Regressor

In [ ]:
lgbm = LGBMRegressor()
       
lgbm.fit(X_train, Y_train)

test_predict = lgbm.predict(X_test)


# LGBM AUC & ROC Curve

In [ ]:
y_pred_proba = lgbm.predict(X_test)
fpr, tpr, _ = metrics.roc_curve(Y_test,  y_pred_proba)
auc = metrics.roc_auc_score(Y_test, y_pred_proba)
print(auc)
plt.plot(fpr,tpr,label="data 1, auc="+str(auc))
plt.legend(loc=4)
plt.show()


# Test Set

In [ ]:
test_set.head()

In [ ]:
test_set.isnull().sum().sort_values(ascending = False)

In [ ]:
test_set = test_set.fillna(test_set.mean()) 

In [ ]:
test_id = test_set['id']
test_x= test_set.drop(columns=['id'])
print(len(test_x.columns))


In [ ]:
y_prdict =xgb.predict_proba(test_x)[:,1]

In [ ]:
submission = pd.DataFrame(list(zip(test_id, y_prdict)),
               columns =['id', 'claim'])
submission.head(20)

In [ ]:
submission.to_csv('submission.csv', index=False)